In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import models,layers
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/completeBhageeraPayload.csv')

In [ ]:
df.head(5)

,mlid,ax,ay,az,gx,gy,gz
0,Walking,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,Walking,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,Walking,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,Walking,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,Walking,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [ ]:
df['mlid'].value_counts() #data is imbalanced

Rest        8141
Sleep       5753
Walking     5241
Running     3698
Standing    1757
Sitting      839
Name: mlid, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
yle = LabelEncoder()
df['mlid'] = yle.fit_transform(df['mlid'])

In [ ]:
df['mlid'].value_counts() 

0    8141
3    5753
5    5241
1    3698
4    1757
2     839
Name: mlid, dtype: int64

In [ ]:
X = df.drop('mlid', axis =1)
Y= df['mlid']

In [ ]:
X.head(5)

,ax,ay,az,gx,gy,gz
0,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [ ]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state = 42)

In [ ]:
# resampling th data
X_resampled , Y_resampled = smk.fit_resample(X, Y)

In [ ]:
Y_resampled.value_counts() #data is now balanced

3    7935
2    7850
4    7830
1    7696
5    7483
0    7340
Name: mlid, dtype: int64

In [ ]:
# Splitting the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_resampled,Y_resampled, test_size=0.2, random_state=0)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
# Convert the labels to one-hot encoded vectors
num_classes = 6 #number of unique classes
y_train_encoded = to_categorical(Y_train, num_classes = num_classes)
y_test_encoded = to_categorical(Y_test, num_classes = num_classes)

In [ ]:
y_train_encoded

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
# Define the architecture of the ANN model
model = Sequential()
model.add(Dense(64, activation = 'relu')) # input_dim represents the number of input features
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))   # Output layer with softmax activation for multiclass classification

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model_history = model.fit(X_train, y_train_encoded, epochs = 100, batch_size = 32)

Epoch 1/100
1154/1154 [==============================] - 4s 2ms/step - loss: 1.0207 - accuracy: 0.5937
Epoch 2/100
1154/1154 [==============================] - 2s 2ms/step - loss: 0.6476 - accuracy: 0.7459
Epoch 3/100
1154/1154 [==============================] - 2s 2ms/step - loss: 0.5699 - accuracy: 0.7819
Epoch 4/100
1154/1154 [==============================] - 2s 2ms/step - loss: 0.5378 - accuracy: 0.7964
Epoch 5/100
1154/1154 [==============================] - 2s 2ms/step - loss: 0.5127 - accuracy: 0.8062
Epoch 6/100
1154/1154 [==============================] - 3s 3ms/step - loss: 0.4898 - accuracy: 0.8163
Epoch 7/100
1154/1154 [==============================] - 3s 2ms/step - loss: 0.4700 - accuracy: 0.8240
Epoch 8/100
1154/1154 [==============================] - 3s 2ms/step - loss: 0.4541 - accuracy: 0.8293
Epoch 9/100
1154/1154 [==============================] - 2s 2ms/step - loss: 0.4361 - accuracy: 0.8355
Epoch 10/100
1154/1154 [==============================] - 2s 2ms/step - l

In [ ]:
print(model_history.history.keys())

dict_keys(['loss', 'accuracy'])


In [ ]:
# Evaluate the model
loss , accuracy = model.evaluate(X_test, y_test_encoded)

289/289 [==============================] - 1s 1ms/step - loss: 0.3488 - accuracy: 0.8876
